In [1]:
import time
import os
from dotenv import load_dotenv

from openai import AzureOpenAI

# gets API Key from environment variable OPENAI_API_KEY
load_dotenv()
api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")


client = AzureOpenAI(api_key=api_key,
api_version=api_version,
azure_endpoint=api_endpoint)

In [2]:
import datetime


def get_flight_details(flight_number):
    url = "https://www.southwest.com/api/air-operations/v1/air-operations/api/air/flights/path"
    headers = {
        "X-Api-Key" : "" # Add your API key here
    }
    request_body = {
        "departureDate": datetime.date.today().strftime("%Y-%m-%d"),
        "flightNumber":{flight_number},
        "application":"air-flight-status",
        "site":"southwest"
    }
    
    return f"""Here is the flight information for flight number {flight_number}:\n
            - The airline name is : Southwest
            - The departure date is : {datetime.date.today().strftime("%Y-%m-%d")}
            - The flight status is : On Time
    """        



In [4]:
get_flight_details(2348)

'Here is the flight information for flight number 2348:\n\n            - The airline name is : Southwest\n            - The departure date is : 2024-02-29\n            - The flight status is : On Time\n    '

In [5]:
assistant = None
print(api_deployment_name)

try:
    assistant = client.beta.assistants.create(
    name="Flight Assistant",
    instructions="You are a flight assistant. If you are asked about a flight, use the flight number user provided and get_flight_details function to get the information about the flight and then answer user's question.",
    tools=[{
        "type": "function",
        "function": {
            "name" : "get_flight_details",
            "description" : "This function takes a flight number as input and returns the flight details.",
            "parameters" : {
                "type" : "object",
                "properties" : {
                    "flight_number" : {
                        "type":"string"
                        }
                    }    
                }
            }
        }],
    model=api_deployment_name,
)
except Exception as e:
    print(e)
assistant    

gpt-4


Assistant(id='asst_h8mlzpuroXC6QxV912F39q52', created_at=1709249332, description=None, file_ids=[], instructions="You are a flight assistant. If you are asked about a flight, use the flight number user provided and get_flight_details function to get the information about the flight and then answer user's question.", metadata={}, model='gpt-4', name='Flight Assistant', object='assistant', tools=[ToolFunction(function=FunctionDefinition(name='get_flight_details', description='This function takes a flight number as input and returns the flight details.', parameters={'type': 'object', 'properties': {'flight_number': {'type': 'string'}}}), type='function')])

In [10]:
thread = client.beta.threads.create()
thread

Thread(id='thread_1Md3cgZBdGbSohoH1Qv3BfxH', created_at=1709249426, metadata={}, object='thread')

In [11]:
run=client.beta.threads.runs.create(
    assistant_id=assistant.id,
    thread_id=thread.id,
    instructions="What is the flight status of flight number 2348?"
)


In [13]:
print("printing run")
while True:
    run = client.beta.threads.runs.retrieve(

        thread_id=thread.id,
        run_id=run.id
    )
    if run.status == "requires_action":
        break
run 

printing run


Run(id='run_V9OXkpvr5s7TYsUTVVKvtUHh', assistant_id='asst_h8mlzpuroXC6QxV912F39q52', cancelled_at=None, completed_at=None, created_at=1709249427, expires_at=1709250027, failed_at=None, file_ids=[], instructions='What is the flight status of flight number 2348?', last_error=None, metadata={}, model='gpt-4', object='thread.run', required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_DahiPSIwzmuVPcjJM6QYkUpb', function=Function(arguments='{\n  "flight_number": "2348"\n}', name='get_flight_details'), type='function')]), type='submit_tool_outputs'), started_at=1709249428, status='requires_action', thread_id='thread_1Md3cgZBdGbSohoH1Qv3BfxH', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='get_flight_details', description='This function takes a flight number as input and returns the flight details.', parameters={'type': 'object', 'properties': {'flight_number': {'type': 'string'}}}), type='fun

In [14]:
tool_calls = run.required_action.submit_tool_outputs.tool_calls
tool_calls

[RequiredActionFunctionToolCall(id='call_DahiPSIwzmuVPcjJM6QYkUpb', function=Function(arguments='{\n  "flight_number": "2348"\n}', name='get_flight_details'), type='function')]

In [16]:
import json
def get_output_for_tool_call(tool_calls):
    flight_number = json.loads(tool_calls.function.arguments)["flight_number"]
    return {
        "tool_call_id" : tool_calls.id,
        "output" :  get_flight_details(flight_number)
    }



In [17]:
tool_outputs = map(get_output_for_tool_call, tool_calls)

In [18]:
tool_outputs = list(tool_outputs)
tool_outputs

[{'tool_call_id': 'call_DahiPSIwzmuVPcjJM6QYkUpb',
  'output': 'Here is the flight information for flight number 2348:\n\n            - The airline name is : Southwest\n            - The departure date is : 2024-02-29\n            - The flight status is : On Time\n    '}]

In [19]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=tool_outputs
)

In [22]:
run.status

'queued'

In [23]:
message = client.beta.threads.messages.list(
    thread_id=thread.id
)

message

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_4WiHfUbMlpQtvUUMaCDwdYcN', assistant_id='asst_h8mlzpuroXC6QxV912F39q52', content=[MessageContentText(text=Text(annotations=[], value='The flight status for flight number 2348 is as follows:\n\n- Airline: Southwest\n- Departure Date: February 29, 2024\n- Flight Status: On Time'), type='text')], created_at=1709249864, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_V9OXkpvr5s7TYsUTVVKvtUHh', thread_id='thread_1Md3cgZBdGbSohoH1Qv3BfxH')], object='list', first_id='msg_4WiHfUbMlpQtvUUMaCDwdYcN', last_id='msg_4WiHfUbMlpQtvUUMaCDwdYcN', has_more=False)